##0. google drive mount와 필요한 모듈 import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw
import numpy as np
import os
import random

##1. data augmentation

In [ ]:
'''
def rotate_coordinates(coords, angle, center)
(4, 2) 형태의 좌표가 들어오면 angle만큼 회전시켜주는 함수입니다.
여기서 center은 (회전된) 이미지의 중심 좌표(x, y)가 들어가야 합니다.
'''
def rotate_coordinates(coords, angle, center):
    # 중심 좌표를 기준으로 회전 각도로 변환
    angle_rad = np.radians(angle)
    cos_theta = np.cos(angle_rad)
    sin_theta = np.sin(angle_rad)

    # 회전 변환 행렬 계산
    rotation_matrix = np.array([[cos_theta, -sin_theta],
                                [sin_theta, cos_theta]])

    # 중심 좌표를 원점으로 변환
    centered_coords = coords - center

    # 회전 변환 적용
    rotated_coords = np.dot(centered_coords, rotation_matrix.T)

    # 원점 좌표로 복원
    restored_coords = rotated_coords + center

    return restored_coords

In [ ]:
'''
def draw_rotated_bbox(image, coords, angle, color=(255, 0, 0), width=2)
check_image 생성을 위한 시각화함수입니다. 기존 image와 bounding box가 결합된 이미지를 반환합니다.
'''

def draw_rotated_bbox(image, coords, angle, color=(255, 0, 0), width=2):
    # PIL 이미지로 변환
    pil_image = Image.fromarray(image)

    # 이미지에 그리기 위한 Draw 객체 생성
    draw = ImageDraw.Draw(pil_image)

    for i in range(0, 4):
        pt1 = tuple(coords[i])
        pt2 = tuple(coords[(i + 1) % 4])
        draw.line([pt1, pt2], fill=color, width=width)

    # PIL 이미지를 다시 NumPy 배열로 변환
    result_image = np.array(pil_image)

    return result_image

In [ ]:
'''
def rotate_image(image, angle)
angle만큼 이미지를 회전합니다.
'''

def rotate_image(image, angle):
    # 이미지의 높이와 너비 추출
    height, width = image.shape[:2]

    # 회전을 위한 변환 행렬 계산
    center = (width // 2, height // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # 이미지 회전
    rotated_image = cv2.warpAffine(image, matrix, (width, height))

    return rotated_image


In [ ]:
'''
def parse_coordinates(line)
문자열을 공백을 기준으로 분리하여 숫자들로 이루어진 리스트를 생성합니다.
라벨링 형식이 다르다면 해당 함수에서 line.split을 수정하면 됩니다.
'''
def parse_coordinates(line):
    coords = line.split(' ')[0:8]
    coords = [float(coord) for coord in coords]

    return coords

In [ ]:
'''
def convert_coordinates(coords_list)
추출된 좌표를 DOTA 형식에 맞게 변환해줍니다. 형시을 변경하고 싶다면, converted_coords 를 정의하는 줄을 수정하면 됩니다.
'''
def convert_coordinates(coords_list):
    converted_coords_list = []
    for coords in coords_list:
        x1, y1 = coords[0]
        x2, y2 = coords[1]
        x3, y3 = coords[2]
        x4, y4 = coords[3]
        classname = "container"
        difficult = 0

        converted_coords = f"{x1} {y1} {x2} {y2} {x3} {y3} {x4} {y4} {classname} {difficult}"
        converted_coords_list.append(converted_coords)

    return converted_coords_list


In [ ]:
'''
def generate_random_values(num_images, num_random_values)
(num_images, num_random_values) 크기로 0~360 사이 랜덤 값이 생성되는 함수입니다.
랜덤 시드 값은 아래와 같이 고정하였습니다.
'''

def generate_random_values(num_images, num_random_values):
    # 랜덤 시드 값을 42로 고정
    random.seed(42)
    np.random.seed(42)

    # [num_images][num_random_values] 크기의 랜덤값 배열 생성
    random_values_array = np.zeros((num_images, num_random_values), dtype=int)

    for i in range(num_images):
        for j in range(num_random_values):
            random_values_array[i, j] = random.randint(0, 360)

    return random_values_array

In [ ]:
'''
def count_files_in_directory(directory_path)
해당 경로에 있는 file 개수를 반환합니다.
'''
def count_files_in_directory(directory_path):
    count = 0
    for filename in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, filename)):
            count += 1
    return count

##2. 함수 호출

본인 경로에 따라 입력 경로와 출력 경로를 적절히 수정합니다.<br>
랜덤 각도에 따라 이미지와 각도를 회전시킵니다.<br>
결과는 images_rotate, labels_rotate에서 확인할 수 있으며, check_images 폴더에서 시각화된 결과를 확인할 수 있습니다.

In [ ]:
# 입력 경로
image_folder_path = "/content/drive/MyDrive/container_dataset_grained/images"
label_folder_path = "/content/drive/MyDrive/container_dataset_grained/labels"

# 출력 경로
output_image_folder_path = "/content/drive/MyDrive/container_dataset_grained/images_rotate"
output_label_folder_path = "/content/drive/MyDrive/container_dataset_grained/labels_rotate"
check_path = "/content/drive/MyDrive/container_dataset_grained/check_images"

os.makedirs(output_image_folder_path, exist_ok=True)
os.makedirs(output_label_folder_path, exist_ok=True)
os.makedirs(check_path, exist_ok=True)


image_files = os.listdir(image_folder_path)

num_images = count_files_in_directory(image_folder_path)
num_random_values = 10

random_values = generate_random_values(num_images, num_random_values)
j = -1

# 이미지와 텍스트 파일 처리 반복문
for image_file in image_files:
  j += 1
  for i in range(0, 10):
    rotate_angle = random_values[j][i]
    # 이미지 경로
    image_path = os.path.join(image_folder_path, image_file)

    # 이미지 읽기
    image = cv2.imread(image_path)
    check_image = cv2.imread(image_path)

    # 이미지 회전
    rotated_image = rotate_image(image, rotate_angle)
    check_image = rotate_image(check_image, rotate_angle)

    # 회전된 이미지 파일 이름 정의
    rotated_image_file = os.path.splitext(image_file)[0] + "_rotate_" + str(i) + '.png'

    # 회전된 이미지 저장
    output_image_path = os.path.join(output_image_folder_path, rotated_image_file)
    cv2.imwrite(output_image_path, rotated_image)

    # 텍스트 파일 경로
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(label_folder_path, label_file)

    # 텍스트 파일 열기
    res_coords = []
    with open(label_path, 'r') as file:
        for line in file:
        # 좌표 파싱해서 numpy로 만들기
          coords = np.array(parse_coordinates(line))
          coords = coords.reshape(4, 2)
          coords = rotate_coordinates(coords, -rotate_angle, center=(rotated_image.shape[1] // 2, rotated_image.shape[0] // 2))
          res_coords.append(coords)
          check_image = draw_rotated_bbox(check_image, coords, rotate_angle)
    converted_coords_list = convert_coordinates(res_coords)
    #print(converted_coords_list)
    check_image_file = os.path.splitext(image_file)[0] + "_rotate_" + str(i) + "_check" + '.png'
    check_image_path = os.path.join(check_path, check_image_file)
    cv2.imwrite(check_image_path, check_image)
    # 회전된 좌표를 파일에 저장
    output_label_file = os.path.splitext(image_file)[0] + "_rotate_" + str(i) + '.txt'
    output_label_path = os.path.join(output_label_folder_path, output_label_file)
    with open(output_label_path, 'w') as file:
      for converted_coords in converted_coords_list:
        file.write(converted_coords + '\n')

  # 완료 메시지 출력
  #print("이미지와 텍스트 파일 처리가 완료되었습니다.")


##3. 확인
파일 개수를 통해 증강이 잘 이루어졌는지 확인합니다.

In [ ]:
# 적절히 증강이 이루어졌는지 확인
file_count = count_files_in_directory(image_folder_path)
print("이미지 파일 개수:", file_count)
file_count = count_files_in_directory(label_folder_path )
print("라벨 파일 개수:", file_count)

file_count = count_files_in_directory(output_image_folder_path)
print("증강 이미지 파일 개수:", file_count)
file_count = count_files_in_directory(output_label_folder_path)
print("증강 라벨 파일 개수:", file_count)
file_count = count_files_in_directory(check_path)
print("체크 이미지 파일 개수:", file_count)